In [ ]:
https://forums.burst-team.us/topic/33/burst-address-generator-v0-2-3-all-platforms
https://github.com/de-luxe/burstcoin-address-generator/releases
get 1 burst coin to create the address in the blockchain
https://faucet.burst-team.us
get the id of BURST-TALE-WK6Y-NMTE-4DB9Z from:
http://burstcoin.biz/address/3430946750618968652 paste the address in the search bar

ew miner:
git clone https://github.com/Mirkic7/mdcct
https://forums.burst-team.us/topic/1142/linux-really-need-help/3
./plotavx2 -k 3430946750618968652 -x 0 -d /tmp/burst/ -s 1 -n 7812000 -m 500 -t 1 -a

wallet:
https://github.com/burst-team/burstcoin/releases

In [ ]:
%reload_ext autoreload
%autoreload 2
from bitmex import *
pl = Poloniex()
getCurrencies()

In [ ]:
# 3-dimensional price cube
import numpy as n
pl = Poloniex()
bars=1000
lsa = 'BTC_ETH BTC_BURST BTC_SC'.split()
rdf = n.zeros([len(lsa),bars,7])
print rdf.shape

fields = 'open high low close quotedVolume volume weightedVolume'.split()
for i in range(len(lsa)):
    df1 = pl.getPoloniexHistorical(symbol=lsa[i], period=14400, bars=bars)
    print df1.shape
    
    rdf[i,:,:] = df1.set_index('date').ix[:,fields].get_values()
#df1.dtypes
#rdf
rf = []
cc = currencyCube(len(lsa),bars,7, index=df1['date'], columns=fields, rdf=rdf)
#cc

In [ ]:
# 3-dimensional price cube
#rdf = currencyCube(550,5,9, index=n.random.randn(550), columns='a b c d e f g h i'.split(' '))
#p.DataFrame(rdf['data'][0], index=rdf['index'], columns=rdf['columns'])
cc = currencyCube(r=len(lsa),tf=bars,c=7, index=df1['date'], columns=fields, rdf=rdf)
cc

In [ ]:
# currency chart overlay
currencyChartOverlay()

In [ ]:
viewHistoricalPricePoloniex()

In [ ]:
viewChartsPoloniex()

In [ ]:
# cypto api 
# https://www.google.com.ar/search?q=coinmarketcap+ohlc&oq=coinmarketcap+ohlc&aqs=chrome..69i57.3815j0j7&client=ubuntu&sourceid=chrome&ie=UTF-8
#https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XMR&start=1405699200&end=9999999999&period=14400
# https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XMR&start=1405699200&end=9999999999&period=14400
    
# poloniex
import drest
api = drest.API('https://poloniex.com/')
response = api.make_request('GET', '/public?command=returnChartData&currencyPair=BTC_XMR&start=1405699200&end=9999999999&period=14400')
print response.data

#df = p.DataFrame(response.data)#.transpose()
#pf(df)
#df

In [ ]:
# cypto api 
df = instrumentIndecesBitmex()

In [ ]:
%reload_ext autoreload
%autoreload 2
from bitmex import *
# coinmarketcap
cmc = CoinMarketCap()
dfc = cmc.tickers()
#with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
#    print dfc
#dfc

In [ ]:
%reload_ext autoreload
%autoreload 2
from bitmex import *
# TokenMarket
tm = TokenMarket()
df = tm.allAssetsBlockchainTokenMarket()
#with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
#    print tm.allAssetsICOsBlockchain

In [ ]:
# TokenMarket
tm.tokenICOsTokenMarket()

In [ ]:
%reload_ext autoreload
%autoreload 2
from bitmex import *
tm.underTheRadarTokens()

In [ ]:
# TokenMarket
tm.tokenmarket()

In [ ]:
%reload_ext autoreload
%autoreload 2
from bitmex import *
cmc = CoinMarketCap()
#cmc.parseCoinMarketCap()

In [ ]:
cmc.getTradableCoins()

In [ ]:
portfolio = cmc.generatePortfolio()
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print portfolio.fillna('')#.ix[:, c]

In [ ]:
# liqui
secret = '<key>'
key = '<secret>'

In [ ]:
def viewLocalbitcoinsCSV(fname):
    dff = p.read_csv(fname)
    with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
        print dff.dtypes
        print dff.fillna('')
viewLocalbitcoinsCSV('/home/qore/qorelogic/assets/sec-svn.git/assets/localbitcoins/Transactions for kiiza.all.csv')

In [ ]:
viewLocalbitcoinsCSV('/home/qore/qorelogic/assets/sec-svn.git/assets/localbitcoins/Transactions for ecambio.all.csv')

In [ ]:
viewLocalbitcoinsCSV('/home/qore/qorelogic/assets/sec-svn.git/assets/localbitcoins/Transactions for bongocoin.all.csv')

In [ ]:
fname = '/home/qore/qorelogic/assets/sec-svn.git/assets/genesis-mining/genesis-mining.payout.jmaloan.log'
#fp = open(fname, 'r')
#res = fp.read()
#fp.close()
dfl = p.read_csv(fname, sep='\t')
#dfl['Mining day 	Currency']
dfl

In [ ]:
%reload_ext autoreload
%autoreload 2
from bitmex import *
# liqui
#print liq.signHMAC512(params)
liq = Liqui(key, secret)
#liq.debugon()
liq.getInfo()
#liq.tradeHistory()
#liq.trade()                                                                                                                                                                                                          

In [ ]:
%reload_ext autoreload
%autoreload 2
from bitmex import *
# bittrex
#print liq.signHMAC512(params)
o = Bittrex('<key>', '<secret>')
#o.debugon()
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    #o.getInfo()
    #o.getDepositAddress('ETH')
    print o.getCurrencies()
    #print o.getMarketSummaries()
    #print o.getMarkets()
    #o.tradeHistory()
    #o.trade()